### Creando un Dashboar para presentación de resultados ###

In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import dash_table
# Cargar los datos en un DataFrame de Pandas
df= pd.read_csv(filepath_or_buffer="D:/Bases_de_datos/input/diabetes.csv")

# Ejemplo de cómo inspeccionar los primeros registros del DataFrame
print(df.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [4]:
## Visión general de los datos con Dash

app = dash.Dash(__name__)

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Div(id='table-container')
])

@app.callback(
    Output('table-container', 'children'),
    [Input('table', 'data')]
)
def display_table(data):
    return html.Div([
        html.P('Datos de la tabla:'),
        html.Pre(str(data))
    ])

if __name__ == '__main__':
    app.run_server(debug=True)


In [8]:
# Asegúrate de tener todas las importaciones necesarias al inicio de tu script
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar el modelo de regresión lineal basado en 'Glucose' y 'Outcome'
X = df[['Glucose']].values.reshape(-1, 1)
y = df['Outcome'].values
model = LinearRegression()
model.fit(X, y)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Análisis Interactivo de Diabetes'),
    dcc.Dropdown(
        id='graph-selector',
        options=[
            {'label': 'Histograma de Glucosa', 'value': 'histogram-glucose'},
            {'label': 'Conteo de Resultados', 'value': 'bar-outcome'},
            {'label': 'Glucosa vs Insulina', 'value': 'scatter-glucose-insulin'},
            {'label': 'BMI vs Edad', 'value': 'scatter-bmi-age'},
            {'label': 'Número de Embarazos', 'value': 'pie-pregnancies'},
            {'label': 'Insulina por Resultado', 'value': 'box-insulin'},
            {'label': 'Regresión Lineal: Outcome vs Glucose', 'value': 'regression-glucose-outcome'}
        ],
        value='histogram-glucose'
    ),
    dcc.Graph(id='selected-graph'),
    html.Div(id='graph-narrative', style={'margin': '20px 0'}),
    html.Label('Filtrar Glucosa:'),
    dcc.RangeSlider(
        id='glucose-filter',
        min=df['Glucose'].min(),
        max=df['Glucose'].max(),
        value=[df['Glucose'].min(), df['Glucose'].max()],
        marks={int(i): str(int(i)) for i in np.linspace(df['Glucose'].min(), df['Glucose'].max(), 10)},
        step=1
    )
])

@app.callback(
    [Output('selected-graph', 'figure'),
     Output('graph-narrative', 'children')],
    [Input('graph-selector', 'value'),
     Input('glucose-filter', 'value')]
)
def update_graph_and_narrative(selected_graph, glucose_range):
    filtered_df = df[(df['Glucose'] >= glucose_range[0]) & (df['Glucose'] <= glucose_range[1])]
    
    # Generar la visualización basada en la selección del usuario
    if selected_graph == 'histogram-glucose':
        figure = px.histogram(filtered_df, x='Glucose', nbins=50, title='Distribución de Niveles de Glucosa')
        narrative = 'Este histograma muestra la distribución de los niveles de glucosa. Es importante para el diagnóstico de diabetes.'
    elif selected_graph == 'bar-outcome':
        figure = px.bar(filtered_df['Outcome'].value_counts(), title='Conteo de Resultados de Diabetes')
        narrative = 'Este gráfico de barras muestra el número de casos positivos y negativos. Ayuda a entender la prevalencia de la enfermedad.'
    elif selected_graph == 'scatter-glucose-insulin':
        figure = px.scatter(filtered_df, x='Glucose', y='Insulin', color='Outcome', title='Glucosa vs Insulina por Resultado')
        narrative = 'Relación entre glucosa e insulina, diferenciada por resultado. Puede indicar cómo la insulina afecta la glucosa en pacientes.'
    elif selected_graph == 'scatter-bmi-age':
        figure = px.scatter(filtered_df, x='BMI', y='Age', color='Outcome', title='BMI vs. Edad por Resultado')
        narrative = 'Este gráfico muestra cómo el BMI y la edad se relacionan con los resultados de diabetes.'
    elif selected_graph == 'pie-pregnancies':
        figure = px.pie(filtered_df, names='Pregnancies', title='Distribución de Número de Embarazos')
        narrative = 'Distribución del número de embarazos entre las pacientes, un factor de riesgo conocido para diabetes tipo 2.'
    elif selected_graph == 'box-insulin':
        figure = px.box(filtered_df, x='Outcome', y='Insulin', color='Outcome', title='Niveles de Insulina por Resultado de Diabetes')
        narrative = 'Comparación de niveles de insulina entre pacientes, segmentado por resultado. La insulina es crucial en el manejo de la diabetes.'
    elif selected_graph == 'regression-glucose-outcome':
        x_range = np.linspace(glucose_range[0], glucose_range[1], 100)
        y_pred = model.predict(x_range.reshape(-1, 1))
        figure = px.scatter(filtered_df, x='Glucose', y='Outcome', opacity=0.65, title='Regresión Lineal: Outcome vs. Glucose')
        figure.add_traces(px.line(x=x_range, y=y_pred, labels={'x': 'Glucose', 'y': 'Predicted Outcome'}).data)
        narrative = 'Modelo de regresión lineal que predice el resultado de la diabetes basado en niveles de glucosa.'
    else:
        figure = {}
        narrative = 'Selecciona un gráfico para visualizar.'
    
    return figure, narrative

if __name__ == '__main__':
    app.run_server(debug=True)
